In [ ]:
!pip install torch transformers sacremoses

In [8]:
import torch
from transformers import *
import pdb
import operator
from collections import OrderedDict
import sys
import traceback
import argparse
import string


import logging

#DEFAULT_MODEL_PATH='bert-large-cased'
#DEFAULT_MODEL_PATH='bert-base-cased' #works best for names
#DEFAULT_MODEL_PATH='bert-base-uncased'
DEFAULT_MODEL_PATH='./'
DEFAULT_TO_LOWER=False
DEFAULT_TOP_K = 20
ACCRUE_THRESHOLD = 1

def init_model(model_path,to_lower):
    logging.basicConfig(level=logging.INFO)
    print("******* MODEL[path] is:",model_path," lower casing is set to:",to_lower)
    tokenizer = BertTokenizer.from_pretrained(model_path,do_lower_case=to_lower)
    model = BertForMaskedLM.from_pretrained(model_path)
    #tokenizer = RobertaTokenizer.from_pretrained(model_path,do_lower_case=to_lower)
    #model = RobertaForMaskedLM.from_pretrained(model_path)
    model.eval()
    return model,tokenizer




def perform_task(model,tokenizer,top_k,accrue_threshold,text,patched):
    text = '[CLS] ' + text + ' [SEP]' 
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    print(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])


    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
        for i in range(len(tokenized_text)):
                #if (i != 0 and i != len(tokenized_text) - 1):
                #    continue
                results_dict = {}
                masked_index = i
                neighs_dict = {}
                if (patched):
                    for j in range(len(predictions[0][0][0,masked_index])):
                        if (float(predictions[0][0][0,masked_index][j].tolist()) > accrue_threshold):
                            tok = tokenizer.convert_ids_to_tokens([j])[0]
                            results_dict[tok] = float(predictions[0][0][0,masked_index][j].tolist())
                        tok = tokenizer.convert_ids_to_tokens([j])[0]
                        if (tok in tokenized_text):
                            neighs_dict[tok] = float(predictions[0][0][0,masked_index][j].tolist())
                else:
                    for j in range(len(predictions[0][0][masked_index])):
                        if (float(predictions[0][0][masked_index][j].tolist()) > accrue_threshold):
                            tok = tokenizer.convert_ids_to_tokens([j])[0]
                            results_dict[tok] = float(predictions[0][0][masked_index][j].tolist())
                        tok = tokenizer.convert_ids_to_tokens([j])[0]
                        if (tok in tokenized_text):
                            neighs_dict[tok] = float(predictions[0][0][masked_index][j].tolist())
                k = 0
                sorted_d = OrderedDict(sorted(results_dict.items(), key=lambda kv: kv[1], reverse=True))
                print("********* Top predictions for token: ",tokenized_text[i])
                for index in sorted_d:
                    if (index in string.punctuation or index.startswith('##') or len(index) == 1 or index.startswith('.') or index.startswith('[')):
                        continue
                    print(index,round(float(sorted_d[index]),4))
                    k += 1
                    if (k > top_k):
                        break
                print("********* Closest sentence neighbors in output to the token :  ",tokenized_text[i])
                sorted_d = OrderedDict(sorted(neighs_dict.items(), key=lambda kv: kv[1], reverse=True))
                for index in sorted_d:
                    if (index in string.punctuation or index.startswith('##') or len(index) == 1 or index.startswith('.') or index.startswith('[')):
                        continue
                    print(index,round(float(sorted_d[index]),4))
                print()
                print()
                #break





Modiy params here

In [11]:

model = "bert-base-cased"
topk = DEFAULT_TOP_K
tolower=False
patched = False
threshold = ACCRUE_THRESHOLD
model,tokenizer = init_model(model,tolower)
text = "Abubakkar Siddiq is a [MASK]"
perform_task(model,tokenizer,topk,threshold,text,patched)

******* MODEL[path] is: bert-base-cased  lower casing is set to: False


loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217

['[CLS]', 'Abu', '##ba', '##kka', '##r', 'Sid', '##di', '##q', 'is', 'a', '[MASK]', '[SEP]']
********* Top predictions for token:  [CLS]
the 4.6188
and 4.2249
of 4.2042
to 3.9931
in 3.9864
on 3.1076
was 3.077
him 3.049
it 3.0144
for 2.9258
her 2.9244
as 2.8541
that 2.8447
families 2.7784
with 2.7692
me 2.6936
is 2.6631
he 2.6377
at 2.6315
by 2.5921
his 2.5727
********* Closest sentence neighbors in output to the token :   [CLS]
is 2.6631
Sid -2.5124
Abu -2.5838


********* Top predictions for token:  Abu
Abu 14.6061
Abdul 8.112
Mu 7.8512
El 6.2221
In 6.1822
Ta 5.9992
in 5.8125
the 5.6926
Abd 5.5745
and 5.5084
of 5.3902
to 5.1345
As 5.041
Ahmed 5.0399
Baba 4.978
Ibrahim 4.8955
Fu 4.8829
The 4.7856
Allah 4.7632
Do 4.6964
Wu 4.6401
********* Closest sentence neighbors in output to the token :   Abu
Abu 14.6061
is 3.7958
Sid 1.316


********* Top predictions for token:  ##ba
Ba 19.1697
Ma 13.368
Baba 12.8254
Bo 11.208
Dana 11.167
Bey 10.0911
ma 9.9483
Na 9.8936
Mira 8.7935
Da 8.6862
Bar 8.